#### CEE 6003 Term Project
#### Hyrum Tennant
Use NAIP imagery to identify surface water in beaver impacted reaches.
Compare data from multiple years to identify increase or decrease in dam construction in given areas by calculating identified surface area of water.

In [325]:
# Initializing display and earthengine
from IPython.display import Image
%matplotlib inline

import ee
ee.Initialize()

import datetime
from matplotlib import dates
import matplotlib.dates as mdates
from pylab import *

# importing ipygee for dynamic mapping
from ipygee import *
# import geetools for cloud /sadow discrimination
from geetools import ui, cloud_mask

Map = Map() # from ipygee
Map.show()

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [326]:
naip = ee.ImageCollection("USDA/NAIP/DOQQ")

Identify areas of interest.

In [327]:
# Logan River
lrArea = ee.Geometry.Rectangle([-111.565907, 41.923843, -111.561849, 41.896613])
# Curtis Creek
ccArea = ee.Geometry.Rectangle([-111.568564, 41.604967, -111.553125,  41.600111])

Get the images for the areas of interest.

In [328]:
#Logan River
# 2011 imagery
# Get the images for the area of interest
LR_11 = naip.filterDate('2011-01-01', '2011-12-31').filterBounds(lrArea)
# mosaic the images for the area of interest
LR_11 = LR_11.mosaic()
# clip the mosaic to the area of interest
LR_11 = LR_11.clip(lrArea)

# 2014 imagery
LR_14 = naip.filterDate('2014-01-01', '2014-12-31').filterBounds(lrArea)
LR_14 = LR_14.mosaic()
LR_14 = LR_14.clip(lrArea)

# Curtis Creek
# 2011 imagery
# Get the images for the area of interest
CC_11 = naip.filterDate('2011-01-01', '2011-12-31').filterBounds(ccArea)
# mosaic the images for the area of interest
CC_11 = CC_11.mosaic()
# clip the mosaic to the area of interest
CC_11 = CC_11.clip(ccArea)

# 2014 imagery
CC_14 = naip.filterDate('2014-01-01', '2014-12-31').filterBounds(ccArea)
CC_14 = CC_14.mosaic()
CC_14 = CC_14.clip(ccArea)

Add color-IR composite images to the Map.

In [329]:
nBands = ['N', 'R', 'G']
Map.addLayer(LR_11, {'bands':nBands,'min': 0, 'max': 255}, name='LR 2011')
Map.addLayer(LR_14, {'bands':nBands,'min': 0, 'max': 255}, name='LR 2014')
Map.addLayer(LR_16, {'bands':nBands,'min': 0, 'max': 255}, name='LR 2016')
Map.addLayer(LR_18, {'bands':nBands,'min': 0, 'max': 255}, name='LR 2018')
Map.addLayer(CC_11, {'bands':nBands,'min': 0, 'max': 255}, name='CC 2011')
Map.addLayer(CC_14, {'bands':nBands,'min': 0, 'max': 255}, name='CC 2014')
Map.addLayer(CC_16, {'bands':nBands,'min': 0, 'max': 255}, name='CC 2016')
Map.addLayer(CC_18, {'bands':nBands,'min': 0, 'max': 255}, name='CC 2018')

Calculate the NDWI for the images.

In [330]:
# function for adding NDWI band to image
def addNDWI(image):
    # add band for NDWI
    return image.\
    addBands(image.expression('((green - NIR)/(green + NIR))', {\
        'NIR': image.select('N'),\
        'green': image.select('G')\
    }).rename('NDWI'))

# set NDWI color palette
ndwiViz = {'bands': ['NDWI'], min: 0.5, max: 1, 'palette': ['00FFFF', '0000FF']}

# Logan River
# process 2011 image
LR_11_ndwi = addNDWI(LR_11)
LR_11_ndwi = LR_11_ndwi.updateMask(LR_11_ndwi.select('NDWI').gte(0.5))
Map.addLayer(LR_11_ndwi, ndwiViz, 'LR_11_ndwi')

# process 2014 image
LR_14_ndwi = addNDWI(LR_14)
LR_14_ndwi = LR_14_ndwi.updateMask(LR_14_ndwi.select('NDWI').gte(0.4))
Map.addLayer(LR_14_ndwi, ndwiViz, 'LR_14_ndwi')


# Curtis Creek
# process 2011 image
CC_11_ndwi = addNDWI(CC_11)
CC_11_ndwi = CC_11_ndwi.updateMask(CC_11_ndwi.select('NDWI').gte(0.5))
Map.addLayer(CC_11_ndwi, ndwiViz, 'CC_11_ndwi')

# process 2014 image
CC_14_ndwi = addNDWI(CC_14)
CC_14_ndwi = CC_14_ndwi.updateMask(CC_14_ndwi.select('NDWI').gte(0.5))
Map.addLayer(CC_14_ndwi, ndwiViz, 'CC_14_ndwi')

Map.centerObject(lrArea)

Calculate the area of water in the images.

In [331]:
area11 = LR_11_ndwi.select(['NDWI']).multiply(ee.Image.pixelArea()).reduceRegion(reducer=ee.Reducer.sum(), geometry=LR_11_ndwi.geometry(), scale=1) # reduceRegion takes keyword arguments, not a dict
print('Logan River 2011')
print(area11.getInfo())

area14 = LR_14_ndwi.select(['NDWI']).multiply(ee.Image.pixelArea()).reduceRegion(reducer=ee.Reducer.sum(), geometry=LR_14_ndwi.geometry(), scale=1) # reduceRegion takes keyword arguments, not a dict
print('Logan River 2014')
print(area14.getInfo())

CCarea11 = CC_11_ndwi.select(['NDWI']).multiply(ee.Image.pixelArea()).reduceRegion(reducer=ee.Reducer.sum(), geometry=CC_11_ndwi.geometry(), scale=1) # reduceRegion takes keyword arguments, not a dict
print('Curtis Creek 2011')
print(CCarea11.getInfo())

CCarea14 = CC_14_ndwi.select(['NDWI']).multiply(ee.Image.pixelArea()).reduceRegion(reducer=ee.Reducer.sum(), geometry=CC_11_ndwi.geometry(), scale=1) # reduceRegion takes keyword arguments, not a dict
print('Curtis Creek 2014')
print(CCarea14.getInfo())

Logan River 2011
{'NDWI': 17424.237132288046}
Logan River 2014
{'NDWI': 15751.729349975474}
Curtis Creek 2011
{'NDWI': 1855.207258633528}
Curtis Creek 2014
{'NDWI': 6659.754359851257}


In 2011 the surface area on the beaver impacted section of the Logan River was 17424 m^2 and 1855 m^2 in the Curtis Creek section.
In 2014 the surface area on the beaver impacted section of the Logan River was 15751 m^2 and 6659 m^2 in the Curtis Creek section.

The large jump observed in the Curtis Creek section was due to the classification of shadows as water. This is an interesting phenomina and adjusting the index value for which water was classified within the NDWI did improve results. It reduced the amount of actual water that was classified as water. 